In [18]:
!pip install requests pandas python-dotenv


In [10]:
class Node:
    def __init__(self, type, value=None, left=None, right=None):
        self.type = type  # 'operator' or 'operand'
        self.value = value  # Operand value or operator (e.g., 'AND')
        self.left = left  # Left child node
        self.right = right  # Right child node


In [12]:
def create_rule(rule_string):
    # Parsing logic can be complex; here we create a simple rule as an example
    if "age" in rule_string and "department" in rule_string:
        return Node('operator', 'AND',
                    Node('operand', {'age': '>30'}),
                    Node('operand', {'department': "'Sales'"}))
    return None


In [6]:
def evaluate_rule(ast, user_data):
    try:
        if ast.type == 'operand':
            key, condition = list(ast.value.items())[0]

            # Handle string comparisons
            if isinstance(user_data[key], str):
                condition_value = condition.strip("'")
                return user_data[key] == condition_value

            # Handle numeric comparisons
            return eval(f"{user_data[key]} {condition}")

        elif ast.type == 'operator':
            if ast.value == 'AND':
                return evaluate_rule(ast.left, user_data) and evaluate_rule(ast.right, user_data)
            elif ast.value == 'OR':
                return evaluate_rule(ast.left, user_data) or evaluate_rule(ast.right, user_data)

        return False
    except KeyError as e:
        print(f"Error: Missing key {e} in user data.")
        return False
    except Exception as e:
        print(f"Error evaluating rule: {e}")
        return False


In [7]:
import sqlite3

# Initialize SQLite database
conn = sqlite3.connect('rules.db')
cursor = conn.cursor()

# Create the table if it doesn't exist
cursor.execute("CREATE TABLE IF NOT EXISTS rules (id INTEGER PRIMARY KEY, rule TEXT)")

# Function to store new rules
def store_rule(rule_string):
    cursor.execute("INSERT INTO rules (rule) VALUES (?)", (rule_string,))
    conn.commit()
    print("New rule stored successfully.")

# Function to modify existing rules
def modify_rule(rule_id, new_rule_string):
    cursor.execute("UPDATE rules SET rule = ? WHERE id = ?", (new_rule_string, rule_id))
    conn.commit()
    print(f"Rule {rule_id} updated successfully.")

# Example usage
store_rule("age > 30 AND department = 'Sales'")  # Store a new rule
modify_rule(1, "age > 25 AND department = 'Marketing'")  # Modify the rule with ID 1


New rule stored successfully.
Rule 1 updated successfully.


In [14]:
# User data and rule
user_data = {'age': 35, 'department': 'Sales'}
rule = create_rule("age > 30 AND department = 'Sales'")

# Evaluate the rule
print(evaluate_rule(rule, user_data))  # Expected Output: True

# Test with different data
user_data = {'age': 28, 'department': 'Marketing'}
rule = create_rule("age > 25 AND department = 'Marketing'")
print(evaluate_rule(rule, user_data))  # Expected Output: True


True
False
